In [14]:
import twitter
import urlparse
from pprint import pprint as pp

class TwitterAPI(object):
    """
    TwitterAPI class allows the Connection to Twitter via OAuth
    once you have registered with Twitter and receive the 
    necessary credentials
    """
    
    # initialize and get the twitter credentials
    def __init__(self):
        consumer_key = '2ttoyoYKzu8XNlh3e3HW2jMNP'
        consumer_secret = 'kRjCiF3NNdyCJQFK1liA0gqTMNEBcf1qPFE3KIlsSj3U8gEk0E'
        access_token = '2853253536-v8zzqsVgPbGPopZ7K1K78HdDAS3ci0gfQ0ZmUyq'
        access_secret = '9RIp81N7Da5zD1V0Zh72VL1TaW16qe51kao8P0WU59Oxk'
        
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret
        self.access_token = access_token
        self.access_secret = access_secret
        
        # authenticate credentials with Twitter us Oauth 
        self.auth = twitter.oauth.OAuth(access_token, access_secret, consumer_key, consumer_secret)
        # creates registered Twitter API
        self.api = twitter.Twitter(auth=self.auth)
    
    # search Twitter with query q and max result
    def searchTwitter(self, q, max_res=10, **kwargs):
        search_results = self.api.search.tweets(q=q, count=10, **kwargs)
        statuses = search_results['statuses']
        max_results  = min(1000, max_res)
        
        for _ in range(10):
            try:
                next_results = search_results['search_metadata']['next_results']
                
            except KeyError as e:
                break
                
            next_results = urlparse.parse_qsl(next_results[1:])
            kwargs = dict(next_results)
            search_results = self.api.search.tweets(**kwargs)
            statuses += search_results['statuses']
            
            if len(statuses) > max_results:
                break
                
        return statuses
                
                
    # parse tweets as it is collected to extract id, creation date, user id, tweet text
    def parseTweets(self, statuses):
        return [(status['id'],
                 status['created_at'],
                 status['user']['id'],
                 status['user']['name'],
                 status['text'], url['expanded_url'])
                    for status in statuses
                    for url in status['entities']['urls']
        ]
        


In [15]:
t = TwitterAPI()
q = "ApacheSpark"
tsearch = t.searchTwitter(q)
tparsed = t.parseTweets(tsearch)

In [18]:
pp(tparsed)

[(772302009403711488,
  u'Sun Sep 04 05:15:22 +0000 2016',
  259226866,
  u'Ryan Nel',
  u'RT @IBMbigdata: #MachineLearning in #ApacheSpark 2.0: Under the Hood and Over the Rainbow https://t.co/0pi21cPxYl https://t.co/lXPbGB3BjX',
  u'http://bit.ly/2ca6lXR'),
 (772300977940549632,
  u'Sun Sep 04 05:11:16 +0000 2016',
  136845362,
  u'Jai',
  u'RT @mapr: #WhiteboardWalkthrough: @ApacheSpark vs. #MapReduce - https://t.co/jbjRhAXtxW https://t.co/dr1eQJtjzy',
  u'http://bit.ly/1T2qQnd'),
 (772298698017509376,
  u'Sun Sep 04 05:02:12 +0000 2016',
  609187397,
  u'Michael Li',
  u'RT @IBMbigdata: #MachineLearning in #ApacheSpark 2.0: Under the Hood and Over the Rainbow https://t.co/BxDESgfBEC https://t.co/XOMd6e96Y4',
  u'http://bit.ly/2cfHOh6'),
 (772290631645356032,
  u'Sun Sep 04 04:30:09 +0000 2016',
  256109210,
  u'Ajay Saini',
  u'RT @ImDataScientist: #apachespark Is the Future of #Hadoop , Cloudera Says #data #BigData #DataScience https://t.co/OAcPkOUlme https://t.co\u2026',
  u'http